# Alumni Education History Filtering

## Imports

In [ ]:
import pandas as pd
import json
import math
import time

## Load Data From File

In [ ]:
edu = pd.read_csv('alum-edu-hist.csv')
edu.head()

## Basic Filtering

### FSB + relevant data filtering

Only get people from FSB

In [ ]:
bus = edu[(edu['Major #1 Department'] == 'Management') | (edu['Major #2 Department'] == 'Management') | (edu['Minor #1 Department'] == 'Management') | (edu['Minor #2 Department'] == 'Management')]
bus.describe()

Select people in the last 50 years

In [ ]:
bus = bus[(bus['Contact: Preferred Class Year'] >= (2020 - 50)) & (bus['Contact: Preferred Class Year'] <= 2020)]
bus.describe()

In [ ]:
bus = bus[bus['Education: Record Type'] == 'WPI Degree']
bus.describe()

Ensure that only relevant degrees are being picked

In [ ]:
bus['Degree/Certificate'].unique()

In [ ]:
accepted_degrees = ['Bachelor of Science', 'Master of Science',
       'Master of Business Admin.', 'Master of Science in Mgmt.',
       'Bachelor of Arts']
accept_regex = '|'.join(accepted_degrees)
bus = bus[bus['Degree/Certificate'].str.contains(accept_regex)]
bus.describe()

### Uniqueness

Seems that we have duplicate names

In [ ]:
bus['Contact: Full Name'].nunique()

Duplicated names are for multiple degrees; should be fine to keep them in

In [ ]:
bus[bus['Contact: Full Name'] == 'Akshay Rao']

## Generate final list of alumni

In [ ]:
names = list(bus['Contact: Full Name'].unique())
names[:10]

Write to a file

In [ ]:
with open('names.json', 'w') as fh:
    json.dump(names, fh)

# WPI Names to LinkedIn URLs

## Imports

In [ ]:
from googleapiclient.discovery import build
from tqdm import tqdm

Enable tqdm pandas mode

In [ ]:
# tqdm.pandas()

## Create Client

In [ ]:
api_key = input('Input your developer key: ')
api_key

In [ ]:
engine_id = input('Input your engine ID: ')
engine_id

In [ ]:
service = build('customsearch', 'v1', developerKey=api_key)

## Create Mapping

Create search query function

In [ ]:
def get_url(row):
    name = row['Contact: Full Name']    
    res = service.cse().siterestrict().list(
       q=f'-intitle:profiles {name}', cx=engine_id, num=1,
       exactTerms='Worcester Polytechnic Institute').execute()
#     res = service.cse().list(
#         q=f'-intitle:profiles {name}', cx=engine_id, num=1,
#         exactTerms='Worcester Polytechnic Institute').execute()
    if res and 'items' in res:
        return res['items'][0]['link']

Reduce the size until we have the final funds

In [ ]:
#limit = 99
#subset = bus[:limit]
subset = bus

Generate all of the urls

In [ ]:
start_time = time.time()
cnt = 0
subset['url'] = None
for _, row in tqdm(subset.iterrows(), total=subset.shape[0]):
    cnt += 1
    subset['url'].iloc[cnt-1] = get_url(row)
    time.sleep(0.6)
        
    if cnt % 250 == 0:
        subset.to_csv('checkpoints/urls-{}.csv'.format(cnt), index=False)

In [ ]:
subset.head()

Write to a file just in case

In [ ]:
subset.to_csv('subset.csv', index=False)

# Scrape LinkedIn

In [ ]:
subset = pd.read_csv('subset.csv')

In [ ]:
subset

Load scraper function

In [ ]:
run scrape/scrape_to_json.py

Scrape Linkedin

In [ ]:
backup = subset

In [ ]:
subset = backup[1000:1100]
subset

In [ ]:
import traceback

scraped = []
cnt = 1
yield_ = tqdm(total=subset.shape[0], desc='Yield')
for url in tqdm(subset[subset['url'].notna()]['url'], total=subset.shape[0], desc='Total'):
    try:
        scraped.append(scrape(url))
        yield_.update(1)
    except:
        traceback.print_exc()
    
    if cnt % 100 == 0:
        with open(f'checkpoints/scrapped-300.json', 'w') as fh:
            fh.write(json.dumps(list(scraped), default=str))
            
    cnt += 1
yield_.close()

In [ ]:
scraped[0]

Save raw data to a file

In [ ]:
with open('scrapped-1000-1100.json', 'w') as fh:
    fh.write(json.dumps(list(scraped), default=str))

# Join and Combine Scraped and Internal Data

In [ ]:
with open('scrapped-1500.json', 'r') as fh:
    scrapped = json.load(fh)

## High Level Data Cleaning

### Fix year / dates
People sometimes don't include the month in their profiles

In [ ]:
def year(info):
    for job in info['professional']:
        if not job['start_year']:
            job['start_year'] = int(job['start_month'])
            job['start_month'] = 'Jan'
        else:
            job['start_year'] = int(job['start_year'])
            
        if not job['end_year']:
            job['end_year'] = int(job['end_month'])
            job['end_month'] = 'Jan'
        else:
            job['end_year'] = int(job['end_year'])
    return info

### Filter Out Incorrect Scrapped Profiles
Right now, the only metric to ensure that the data is valid is if they went to WPI 

In [ ]:
data = []
for s in list(scrapped):
    went_to_wpi = False
    for edu in s['academic']:
        if edu['name'] == 'Worcester Polytechnic Institute':
           # data.append(year(s))
            data.append(s)

len(data)

## Create Derived Features
Create the features first, then assign them one at a time

### Years Since First Job
Show how long the person spent in each job

In [ ]:
def years_since_first_job(info):
    if not info['professional']:
        return info
    
    first_job = info['professional'][0]['start_year']
    
    for job in info['professional']:
        # HACK NEED TO FIX
        job['years_since_start'] = job['start_year'] - first_job
    
    return info

### Job Tenure
How long the person has been at their current job.

In [ ]:
def job_tenure(info):
    for job in info['professional']:
        # HACK NEED TO FIX
        job['tenure'] = max(job['end_year'] - job['start_year'], 1)
    return info

### Alunni Education History Join
Join the LinkedIn data against WPI's internal database. In terms of conflict, always prefer WPI's database to the LinkedIn Data

In [ ]:
def merge_edu_history(info):
    history = subset.loc[subset['url'] == info['url']]
    
    # Delete any WPI data
    edu_backup = info['academic'].copy()
    joint_edu = []
    for edu in edu_backup:
        if edu['name'] != 'Worcester Polytechnic Institute':
            joint_edu.append(edu)
                
    # Join with WPI Data
    for i in range(history.shape[0]):
        degree = history.iloc[i].fillna('None')
        
        joint_edu.append({
            'name': 'Worcester Polytechnic Institute',
            'type': degree['Degree/Certificate'],
            'start': degree['Start Year'],
            'end': degree['Degree Year'],
            'major_1': degree['Major #1'],
            'major_1_dept': degree['Major #1 Department'],
            'major_2': degree['Major #2'],
            'major_2_dept': degree['Major #2 Department'],
            'minor_1': degree['Minor #1'],
            'minor_1_dept': degree['Minor #1 Department'],
            'minor_2': degree['Minor #2'],
            'minor_2_dept': degree['Minor #2 Department'],
        })
            
    # print(joint_edu)        
    # edu_hist = sorted(joint_edu, key=lambda x: int(x['end']))
    edu_hist = joint_edu
    for i, edu in enumerate(edu_hist):
        edu['order'] = i
    
    info['academic'] = edu_hist
    return info

In [ ]:
merge_edu_history(data[0])

## Merge all modifications

Create a super simple, parallelizable function

In [ ]:
def post(d):
    d = years_since_first_job(d)
    d = job_tenure(d)
    d = merge_edu_history(d)
    return d

# Final Join
Merge all of the data together and export

In [ ]:
final = [merge_edu_history(d) for d in data]

Write and save to a file

In [ ]:
with open('final.json', 'w') as fh:
    fh.write(json.dumps(final))